In [27]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Instalar Framework de agentes do Google
!pip install -q google-adk

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)

import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown, SVG
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

################################################
# --- Agente 1: Agente de contextualização --- #
################################################
def context_agent(topico, process_document):
    context_agent = Agent(
        name="context_agent",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente
        instruction="""
          - Papel: Focado em obter e detalhar o contexto do problema.
          - Funções:
            - Coletar informações sobre o cenário, histórico e ambiente do problema.
            - Identificar as partes interessadas relevantes.
            - Analisar as interdependências e restrições contextuais.
          - Saída: Contexto detalhado e estruturado do problema.
        """,
        description="Agente de contextualização",
        tools=[google_search]
    )

    context_agent_input = f"Documento do processo: {' '.join(process_document)}"

    context = call_agent(context_agent, context_agent_input)
    return context

######################################################
# --- Agente 2: Agente de Definição de Objetivos --- #
######################################################
def goal_agent(topico, process_document):
    goal_agent = Agent(
        name="goal_agent",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente
        instruction="""
          - Papel: Responsável por elicitar e formalizar os objetivos do planejamento.
          - Funções:
            - Obter a declaração do objetivo geral do usuário.
            - Decompor o objetivo geral em sub-objetivos SMART.
            - Priorizar os objetivos com base na importância e urgência.
            - Resolver conflitos entre objetivos, se existirem.
          - Saída: Conjunto de objetivos SMART e priorizados.
        """,
        description="Agente de Definição de Objetivos",
        tools=[google_search]
    )

    goal_agent_input = f"Documento do processo: {' '.join(process_document)}"

    goal = call_agent(goal_agent, goal_agent_input)
    return goal

##################################################
# --- Agente 3: Agente de Análise de Decisão --- #
##################################################
def decision_agent(topico, process_document):
    decision_agent = Agent(
        name="decision_agent",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente
        instruction="""
          - Papel: Especializado em determinar o tipo de decisão a ser tomada e os critérios para avaliá-la.
          - Funções:
            - Identificar o tipo de decisão (e.g., escolha, classificação, previsão, otimização).
            - Definir os critérios de sucesso para a decisão.
            - Estabelecer as métricas para avaliar as alternativas de decisão.
          - Saída: Tipo de decisão e critérios de avaliação definidos.
        """,
        description="Agente de Análise de Decisão",
        tools=[google_search]
    )

    decision_agent_input = f"Documento do processo: {' '.join(process_document)}"

    decision = call_agent(decision_agent, decision_agent_input)
    return decision

#########################################################
# --- Agente 4: Agente de Especificação de Detalhes --- #
#########################################################
def detail_agent(topico, process_document):
    detail_agent = Agent(
        name="detail_agent",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente
        instruction="""
          - Papel: Encarregado de determinar o nível de detalhe necessário para o plano.
          - Funções:
            - Avaliar a complexidade do problema.
            - Considerar as necessidades de informação do usuário.
            - Especificar a profundidade do plano (alto nível ou detalhado).
          - Saída: Nível de detalhe do plano especificado.
        """,
        description="Agente de Especificação de Detalhes",
        tools=[google_search]
    )

    detail_agent_input = f"Documento do processo: {' '.join(process_document)}"

    detail = call_agent(detail_agent, detail_agent_input)
    return detail

#####################################################
# --- Agente 5: Agente de Análise de Restrições --- #
#####################################################
def constraint_agent(topico, process_document):
    constraint_agent = Agent(
        name="constraint_agent",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente
        instruction="""
          - Papel: Dedicado a identificar e analisar as restrições e limitações do planejamento.
          - Funções:
            - Coletar informações sobre restrições de recursos (tempo, orçamento, pessoal).
            - Identificar restrições de dados (disponibilidade, qualidade).
            - Analisar restrições legais, éticas e outras.
          - Saída: Conjunto de restrições analisadas e documentadas.
        """,
        description="**Agente de Análise de Restrições",
        tools=[google_search]
    )

    constraint_agent_input = f"Documento do processo: {' '.join(process_document)}"

    constraint = call_agent(constraint_agent, constraint_agent_input)
    return constraint

###############################################
# --- Agente 6: Agente de Coleta de Dados --- #
###############################################
def data_colletion_agent(topico, process_document):
    data_colletion_agent = Agent(
        name="data_colletion_agent",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente
        instruction="""
          - Papel: Responsável por identificar e obter os dados relevantes para o planejamento.
          - Funções:
            - Identificar as fontes de dados necessárias.
            - Definir os métodos de coleta de dados.
            - Implementar a coleta de dados (ou supervisionar o processo).
          - Saída: Dados coletados e organizados.
        """,
        description="Agente de Coleta de Dados",
        tools=[google_search]
    )

    data_colletion_agent_input = f"Documento do processo: {' '.join(process_document)}"

    data_colletion = call_agent(data_colletion_agent, data_colletion_agent_input)
    return data_colletion

###################################################
# --- Agente 7: Agente de Tratamento de Dados --- #
###################################################
def data_processing_agent(topico, process_document):
    data_processing_agent = Agent(
        name="data_processing_agent",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente
        instruction="""
          - Papel: Especializado em preparar os dados para análise.
          - Funções:
            - Limpar os dados (remover erros, valores ausentes, etc.).
            - Transformar os dados (normalizar, agregar, etc.).
            - Integrar dados de diferentes fontes.
          - Saída: Dados tratados e prontos para análise.

        """,
        description="Agente de Tratamento de Dados",
        tools=[google_search]
    )

    data_processing_agent_input = f"Documento do processo: {' '.join(process_document)}"

    data_processing = call_agent(data_processing_agent, data_processing_agent_input)
    return data_processing

################################################
# --- Agente 8: Agente de Análise de Dados --- #
################################################
def data_analysis_agent(topico, process_document):
    data_analysis_agent = Agent(
        name="data_analysis_agent",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente
        instruction="""
          - Papel: Encarregado de realizar a análise dos dados para extrair insights relevantes para o planejamento.
          - Funções:
            - Selecionar as técnicas de análise apropriadas (estatística descritiva, inferencial, aprendizado de máquina, etc.).
            - Aplicar as técnicas de análise aos dados.
            - Interpretar os resultados da análise.
          - Saída: Insights e informações relevantes para o planejamento.
        """,
        description="Agente de Análise de Dados",
        tools=[google_search]
    )

    data_analysis_agent_input = f"Documento do processo: {' '.join(process_document)}"

    data_analysis = call_agent(data_analysis_agent, data_analysis_agent_input)
    return data_analysis

##########################################################
# --- Agente 9: Agente de Avaliação de Alternativas --- #
##########################################################
def alternatives_agent(topico, process_document):
    alternatives_agent = Agent(
        name="alternatives_agent",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente
        instruction="""
          - Papel: Dedicado a gerar e avaliar diferentes alternativas de ação.
          - Funções:
            - Gerar diferentes opções para alcançar os objetivos.
            - Avaliar as alternativas com base nos critérios definidos.
            - Analisar os riscos e benefícios de cada alternativa.
          - Saída: Avaliação comparativa das alternativas.
        """,
        description="Agente de Avaliação de Alternativas",
        tools=[google_search]
    )

    alternatives_agent_input = f"Documento do processo: {' '.join(process_document)}"

    alternatives = call_agent(alternatives_agent, alternatives_agent_input)
    return alternatives

##########################################
# --- Agente 10: Agente de Relatório --- #
##########################################
def report_agent(topico, process_document):
    report_agent = Agent(
        name="report_agent",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente
        instruction="""
          - Papel: Responsável por gerar o relatório da proposta de planejamento.
          - Funções:
            - Estruturar o relatório com base nas informações dos outros agentes.
            - Redigir o texto do relatório de forma clara e concisa.
            - Incluir os modelos e visualizações gerados pelo Agente de Modelagem.
          - Saída: Relatório da Proposta de Planejamento.
        """,
        description="Agente de Relatório",
        tools=[google_search]
    )

    report_agent_input = f"Documento do processo: {' '.join(process_document)}"

    report = call_agent(report_agent, report_agent_input)
    return report

#############################################
# --- Agente 11: Agente de Formulário --- #
#############################################
def form_agent(topico, process_document):
    form_agent = Agent(
        name="form_agent",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente
        instruction="""
          - Papel: Especializado em gerar o formulário de monitoramento de progresso.
          - Funções:
            - Definir a estrutura e os campos do formulário.
            - Incluir os elementos necessários para acompanhar o progresso do plano.
            - Garantir a consistência com o relatório de planejamento.
          - Saída: Formulário de Monitoramento de Progresso.
        """,
        description="Agente de Formulário",
        tools=[google_search]
    )

    form_agent_input = f"Documento do processo: {' '.join(process_document)}"

    form = call_agent(form_agent, form_agent_input)
    return form

#############################################
# --- Agente 12: Agente de Apresentação --- #
#############################################
def presentation_agent(topico, process_document):
    presentation_agent = Agent(
        name="presentation_agent",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente
        instruction="""
          - Papel: Encarregado de formatar e apresentar os documentos de saída de forma eficaz.
          - Funções:
            - Formatar o relatório e o formulário para facilitar a leitura e a compreensão.
            - Gerar resumos e visualizações dos principais resultados (se necessário).
            - Adaptar a apresentação ao público-alvo (se aplicável).
          - Saída: Relatório e formulário formatados e apresentados.
        """,
        description="Agente de Apresentação",
        tools=[google_search]
    )

    presentation_agent_input = f"Documento do processo: {' '.join(process_document)}"

    presentation = call_agent(presentation_agent, presentation_agent_input)
    return presentation

# Função de laço da chamada de agente
def call_agent_loop(agent, topico, process_document):
  satisfied = False
  while satisfied is False:
      agent_process = agent(topico, process_document)
      print(f"\n--- 📝 Resultado do Agente {agent.__name__.replace('_agent', '')} ---\n")
      display(to_markdown(agent_process))
      print("--------------------------------------------------------------")

      user_response = input("\n  Digite sim para continuar para a próxima etapa; \n   ou complemente a entrada anterior: ")

      if user_response.lower() == "sim":
          process_document.append(agent_process)
          satisfied = True
      else:
          complemento = user_response
          process_document.append(agent_process + " Complemento: " + user_response)

# incialização de variáveis
context = "Esperando o usuario dar mais detalhes, se necessário"
process_document = []
process_document.append(context)

In [ ]:
print("🚀 Iniciando o Sistema de Planejamento de Projetos com 13 Agentes 🚀")

#Obter o Tópico do Usuário
topico = input("❓ Por favor, digite o projeto sobre o qual você quer criar um planejamento: ")

#lógica do sistema de agentes
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o planejamento do projeto para: {topico}")

process_document.append(topico)
call_agent_loop(context_agent, topico, process_document)
call_agent_loop(goal_agent, topico, process_document)
call_agent_loop(decision_agent, topico, process_document)
call_agent_loop(detail_agent, topico, process_document)
call_agent_loop(constraint_agent, topico, process_document)
call_agent_loop(data_colletion_agent, topico, process_document)
call_agent_loop(data_processing_agent, topico, process_document)
call_agent_loop(data_analysis_agent, topico, process_document)
call_agent_loop(alternatives_agent, topico, process_document)
call_agent_loop(report_agent, topico, process_document)
call_agent_loop(form_agent, topico, process_document)
call_agent_loop(presentation_agent, topico, process_document)